In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

diabetes = pd.read_csv('C:/Users/админ/Desktop/IDE/SKILLFACTORY/PYTHON 14/Data/diabetes_data.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
0,6,98,58,33,190,34.0,0.430,43,0,Female
1,2,112,75,32,0,35.7,0.148,21,0,Female
2,2,108,64,0,0,30.8,0.158,21,0,Female
3,8,107,80,0,0,24.6,0.856,34,0,Female
4,7,136,90,0,0,29.9,0.210,50,0,Female


In [3]:
# 8.1 Найдите все повторяющиеся строки в данных и удалите их
dupl_columns = list(diabetes.columns)

mask = diabetes.duplicated(subset=dupl_columns)
diabetes_duplicates = diabetes[mask]
print(f'Число найденных дубликатов: {diabetes_duplicates.shape[0]}')

diabetes_dedupped = diabetes.drop_duplicates(subset=dupl_columns)
print(f'Результирующее число записей: {diabetes_dedupped.shape[0]}')


Число найденных дубликатов: 10
Результирующее число записей: 768


In [5]:
# 8.2 найдите все неинформативные признаки в данных и избавьтесь от них
# удалите все признаки, для которых 95 % значений повторяются или 95 % записей уникальны
low_information_cols = [] 

for col in diabetes_dedupped.columns:
    top_freq = diabetes_dedupped[col].value_counts(normalize=True).max()
    nunique_ratio = diabetes_dedupped[col].nunique() / diabetes_dedupped[col].count()
    if top_freq > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')
        
diabetes_dedupped = diabetes_dedupped.drop(low_information_cols, axis=1)
print(f'Результирующее число признаков: {diabetes_dedupped.shape[1]}')


Gender: 100.0% одинаковых значений
Результирующее число признаков: 9


In [6]:
# 8.3 Замените все записи, равные 0, в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI на символ пропуска
diabetes_dedupped['Glucose'] = diabetes_dedupped['Glucose'].apply(lambda x: x if x != 0 else np.nan)
diabetes_dedupped['BloodPressure'] = diabetes_dedupped['BloodPressure'].apply(lambda x: x if x != 0 else np.nan)
diabetes_dedupped['SkinThickness'] = diabetes_dedupped['SkinThickness'].apply(lambda x: x if x != 0 else np.nan)
diabetes_dedupped['Insulin'] = diabetes_dedupped['Insulin'].apply(lambda x: x if x != 0 else np.nan)
diabetes_dedupped['BMI'] = diabetes_dedupped['BMI'].apply(lambda x: x if x != 0 else np.nan)
cols_null_percent = diabetes_dedupped.isnull().mean()
display(round(cols_null_percent,2))


Pregnancies                 0.00
Glucose                     0.01
BloodPressure               0.05
SkinThickness               0.30
Insulin                     0.49
BMI                         0.01
DiabetesPedigreeFunction    0.00
Age                         0.00
Outcome                     0.00
dtype: float64

In [10]:

# 8.4 далите из данных признаки, где число пропусков составляет более 30 %
thresh = diabetes_dedupped.shape[0]*0.7
diabetes_dedupped = diabetes_dedupped.dropna(thresh=thresh, axis=1)
display(diabetes_dedupped.shape[1])

8

In [11]:

# 8.5 Удалите из данных только те строки, в которых содержится более двух пропусков одновременно
m = diabetes_dedupped.shape[1]
diabetes_dedupped =diabetes_dedupped.dropna(thresh=m-2, axis=0)
print(diabetes_dedupped.shape[0])
diabetes_dedupped.info()

761
<class 'pandas.core.frame.DataFrame'>
Int64Index: 761 entries, 0 to 767
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               761 non-null    int64  
 1   Glucose                   756 non-null    float64
 2   BloodPressure             733 non-null    float64
 3   SkinThickness             541 non-null    float64
 4   BMI                       757 non-null    float64
 5   DiabetesPedigreeFunction  761 non-null    float64
 6   Age                       761 non-null    int64  
 7   Outcome                   761 non-null    int64  
dtypes: float64(5), int64(3)
memory usage: 53.5 KB


In [12]:
# В оставшихся записях замените пропуски на медиану. Чему равно среднее значение в столбце SkinThickness? Ответ округлите до десятых.
#создаём словарь 'имя_столбца': число (признак), на который надо заменить пропуски 
values = {
    'Pregnancies': diabetes_dedupped['Pregnancies'].median(),
    'Glucose': diabetes_dedupped['Glucose'].median(),
    'BloodPressure': diabetes_dedupped['BloodPressure'].median(),
    'SkinThickness': diabetes_dedupped['SkinThickness'].median(),
    'BMI': diabetes_dedupped['BMI'].median(),
    'DiabetesPedigreeFunction': diabetes_dedupped['DiabetesPedigreeFunction'].median(),
    'Age': diabetes_dedupped['Age'].median(),
    'Outcome': diabetes_dedupped['Outcome'].median(),
}
#заполняем оставшиеся записи константами в соответствии со словарем values
diabetes_dedupped = diabetes_dedupped.fillna(values)
#выводим результирующую долю пропусков
display(diabetes_dedupped.isnull().mean())

Pregnancies                 0.0
Glucose                     0.0
BloodPressure               0.0
SkinThickness               0.0
BMI                         0.0
DiabetesPedigreeFunction    0.0
Age                         0.0
Outcome                     0.0
dtype: float64

In [16]:
diabetes_dedupped['SkinThickness'].mean().round(1)

29.1

In [17]:
# 8.7 Сколько выбросов найдёт классический метод межквартильного размаха в признаке SkinThickness?

def outliers_iqr(data, feature):
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    outliers = data[(x<lower_bound) | (x > upper_bound)]
    cleaned = data[(x>lower_bound) & (x < upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_iqr(diabetes_dedupped, 'SkinThickness')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу Тьюки: 87
Результирующее число записей: 674


In [19]:
# 8.8 Сколько выбросов найдёт классический метод z-отклонения в признаке SkinThickness?
def outliers_z_score(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - 3 * sigma
    upper_bound = mu + 3 * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned
outliers, cleaned = outliers_z_score(diabetes_dedupped, 'SkinThickness', log_scale=False)
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу z-отклонения: 4
Результирующее число записей: 757


In [20]:
# 8.9 Найдите сначала число выбросов в признаке DiabetesPedigreeFunction с помощью классического метода межквартильного размаха. 
# Затем найдите число выбросов в этом же признаке в логарифмическом масштабе (при логарифмировании единицу прибавлять не нужно!). 
# Какова разница между двумя этими числами (вычтите из первого второе)?
def outliers_iqr(data, feature):
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    outliers = data[(x<lower_bound) | (x > upper_bound)]
    cleaned = data[(x>lower_bound) & (x < upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_iqr(diabetes_dedupped, 'DiabetesPedigreeFunction')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу Тьюки: 29
Результирующее число записей: 732


In [22]:
def outliers_z_score(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - 3 * sigma
    upper_bound = mu + 3 * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned
outliers, cleaned = outliers_z_score(diabetes_dedupped, 'DiabetesPedigreeFunction', log_scale=True)
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу z-отклонения: 0
Результирующее число записей: 761
